In [1]:
import pymongo

In [2]:
from pyspark.sql import SparkSession

my_spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1')\
    .getOrCreate()
#     .config("spark.mongodb.input.uri", "mongodb://localhost:27017/Tags.Reviews?readPreference=primaryPreferred") \
#     .config("spark.mongodb.output.uri", "mongodb://localhost:27017/Tags.Reviews") \


In [3]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient.get_database('Tags')
tags_col = mydb.get_collection('Reviews')

mydb = myclient["Tags"]

In [6]:
import boto3
from boto.s3.connection import S3Connection
import boto
from boto.s3.key import Key
import sys
import os
import json
import pandas as pd
from collections import defaultdict
# import pyplot as plt
import ast 
%matplotlib inline
sys.path.insert(0,os.path.abspath('..'))
from utils.credentials import access_key, secret_access_key

client = boto3.client('s3')

conn = boto.s3.connect_to_region('us-east-2',
       aws_access_key_id=access_key,
       aws_secret_access_key=secret_access_key,
       is_secure=True,
       calling_format = boto.s3.connection.OrdinaryCallingFormat(),
       )

bucket = conn.get_bucket('persproj')

In [7]:
%%time
food_review_chunks = pd.read_csv('s3://persproj/data/food_review_df.csv')

CPU times: user 2min 28s, sys: 1min 29s, total: 3min 58s
Wall time: 22min 40s


In [9]:
food_review_df = food_review_chunks  

In [10]:
pwd

'/Users/miraekim/workspace/coursework/persproj/eda'

In [ ]:
DATASET_FILE = '/Users/miraekim/workspace/coursework/persproj/topic_modeling_vlad/yelp_dataset/review.json'
MONGO_CONNECTION_STRING = "mongodb://127.0.0.1:27017/"
REVIEWS_DATABASE = "Dataset_Challenge_Reviews"
TAGS_DATABASE = "Tags"
REVIEWS_COLLECTION = "Reviews"
TAGS_COLLECTION = "Tags"
CORPUS_COLLECTION = "Corpus"

food_review_set = set(food_review_df['review_id'].values)
food_review_df = food_review_df.set_index('review_id')

In [13]:
food_review_df[['cool', 'funny','useful','is_open']] = food_review_df[['cool', 'funny','useful','is_open']].astype(float).fillna(0).astype(int)

In [14]:
food_review_df = food_review_df.astype(str)

In [15]:
food_review_df.head()
# .loc['wB16AMXL9JWzy9Dkl1dF1g']['cool']

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,...,review_count,business_stars,state,cool,date,funny,review_stars,text,useful,user_id
review_id,,,,,,,,,,,,,,,,,,,,,
6W0MQHmasK0IsaoDo4bmkw,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605498974300005,-79.65228890899999,Emerald Chinese Restaurant,...,128,2.5,ON,0,2017-01-27 21:54:30,2,3.0,My girlfriend and I went for dinner at Emerald...,3,2K62MJ4CJ19L8Tp5pRfjfQ
BeeBfUxvzD4qNX4HxrgA5g,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605498974300005,-79.65228890899999,Emerald Chinese Restaurant,...,128,2.5,ON,0,2013-06-24 23:11:30,0,3.0,We've always been there on a Sunday so we were...,0,A0kENtCCoVT3m7T35zb2Vg
A1D2kUnZ0HTroFreAheNSg,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605498974300005,-79.65228890899999,Emerald Chinese Restaurant,...,128,2.5,ON,0,2016-01-04 12:59:22,0,3.0,"***No automatic doors, not baby friendly!*** I...",0,SuOLY03LW5ZcnynKhbTydA
2pf45Stf-pNew-xgTababQ,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605498974300005,-79.65228890899999,Emerald Chinese Restaurant,...,128,2.5,ON,0,2014-05-09 02:38:43,0,1.0,"Horrible service,\nI went there tonight with m...",1,lymyUak6KNcNKoDbK87MiQ
RHhlmL07evgAdPaXQV8Omg,30 Eglinton Avenue W,"{'RestaurantsReservations': 'True', 'GoodForMe...",QXAEGFB4oINsVuTFxEYKFQ,"Specialty Food, Restaurants, Dim Sum, Imported...",Mississauga,"{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...",1,43.605498974300005,-79.65228890899999,Emerald Chinese Restaurant,...,128,2.5,ON,2,2011-03-21 14:39:55,1,4.0,One of the gauges of a good Chinese restaurant...,2,6vU0I5XgCv9OQHZ76rV6qw


In [54]:
food_review_dict = food_review_df.to_dict(orient='index')

In [55]:
food_review_dict.get('RHhlmL07evgAdPaXQV8Omg')

{'address': '30 Eglinton Avenue W',
 'attributes': '{\'RestaurantsReservations\': \'True\', \'GoodForMeal\': "{\'dessert\': False, \'latenight\': False, \'lunch\': True, \'dinner\': True, \'brunch\': False, \'breakfast\': False}", \'BusinessParking\': "{\'garage\': False, \'street\': False, \'validated\': False, \'lot\': True, \'valet\': False}", \'Caters\': \'True\', \'NoiseLevel\': "u\'loud\'", \'RestaurantsTableService\': \'True\', \'RestaurantsTakeOut\': \'True\', \'RestaurantsPriceRange2\': \'2\', \'OutdoorSeating\': \'False\', \'BikeParking\': \'False\', \'Ambience\': "{\'romantic\': False, \'intimate\': False, \'classy\': False, \'hipster\': False, \'divey\': False, \'touristy\': False, \'trendy\': False, \'upscale\': False, \'casual\': True}", \'HasTV\': \'False\', \'WiFi\': "u\'no\'", \'GoodForKids\': \'True\', \'Alcohol\': "u\'full_bar\'", \'RestaurantsAttire\': "u\'casual\'", \'RestaurantsGoodForGroups\': \'True\', \'RestaurantsDelivery\': \'False\'}',
 'business_id': 'QXAEG

### Parallel

In [141]:
import multiprocessing
import time
import sys
import nltk
import os
import time

from pymongo import MongoClient
from nltk.stem.wordnet import WordNetLemmatizer
TAGS_COLLECTION = "Tags"

reviews_collection = MongoClient(MONGO_CONNECTION_STRING)[TAGS_DATABASE][REVIEWS_COLLECTION]
corpus_collection = MongoClient(MONGO_CONNECTION_STRING)[TAGS_DATABASE][CORPUS_COLLECTION]

review_cursor = reviews_collection.find()
reviewsCount = review_cursor.count()
print('total reviews:', reviewsCount)

lem = WordNetLemmatizer()

done = 0
start = time.time()


def worker(identifier, skip, count):
    done = 0
    start = time.time()

    reviews_collection = MongoClient(MONGO_CONNECTION_STRING)[TAGS_DATABASE][REVIEWS_COLLECTION]

    batch_size = 100
    for batch in range(0, count, batch_size):
        reviews_cursor = reviews_collection.find().skip(skip + batch).limit(batch_size)
        for review in reviews_cursor:
            food_review_record = food_review_dict.get(review['reviewId'])
            if food_review_record:
#                 nouns = set([])
                nouns = {word['word'] for word in review["words"] if word["pos"] in {"NN", "NNS"}}

#                 for word in words:
#                     nouns.add(lem.lemmatize(word["word"]))
#                 print('finish word set')

                corpus_collection.insert({
                    "reviewId": review["reviewId"],
                    "business": review["business"],
                    "text": review["text"],
                    "words": list(nouns),
                    "hours": food_review_record["hours"],
                    "address": food_review_record["address"],
                    "attributes":food_review_record["attributes"],
                    "categories": food_review_record["categories"],
                    "city": food_review_record["city"],
                    "is_open":food_review_record["is_open"],
                    "latitude": food_review_record["latitude"],
                    "longitude": food_review_record["longitude"],
                    "name": food_review_record["name"],
                    "postal_code":food_review_record["postal_code"],
                    "review_count":food_review_record["review_count"],
                    "business_stars":food_review_record["business_stars"],
                    "state":food_review_record["state"],
                    "cool": food_review_record["cool"],
                    "date": food_review_record["date"],
                    "funny": food_review_record["funny"],
                    "review_stars": food_review_record["review_stars"],
                    "useful":food_review_record["useful"],
                    "user_id":food_review_record["user_id"]
                })

            done += 1
            if done % 1000 == 0:
                end = time.time()
                os.system('cls')
                print('Worker' + str(identifier) + ': Done ' + str(done) + ' out of ' + str(count) + ' in ' + (
                    "%.2f" % (end - start)) + ' sec ~ ' + ("%.2f" % (done / (end - start))) + '/sec')


def main():
    workers = 5
    batch = reviewsCount // workers
    left = reviewsCount % workers

    jobs = []
    for i in range(workers):
        size = reviewsCount // workers
        if i == (workers - 1):
            size += left
        p = multiprocessing.Process(target=worker, args=((i + 1), i * batch, size))
        jobs.append(p)
        p.start()

    for j in jobs:
        j.join()
        print('%s.exitcode = %s' % (j.name, j.exitcode))


/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:16: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  app.launch_new_instance()


total reviews: 6516900


In [ ]:
main()

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:67: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Worker1: Done 1000 out of 1303380 in 32.82 sec ~ 30.47/sec
Worker1: Done 2000 out of 1303380 in 51.46 sec ~ 38.87/sec
Worker1: Done 3000 out of 1303380 in 74.16 sec ~ 40.45/sec
Worker1: Done 4000 out of 1303380 in 86.71 sec ~ 46.13/sec
Worker1: Done 5000 out of 1303380 in 102.97 sec ~ 48.56/sec
Worker1: Done 6000 out of 1303380 in 114.17 sec ~ 52.55/sec
Worker1: Done 7000 out of 1303380 in 140.57 sec ~ 49.80/sec
Worker1: Done 8000 out of 1303380 in 153.68 sec ~ 52.06/sec
Worker1: Done 9000 out of 1303380 in 164.11 sec ~ 54.84/sec
Worker1: Done 10000 out of 1303380 in 173.69 sec ~ 57.57/sec
Worker1: Done 11000 out of 1303380 in 183.56 sec ~ 59.93/sec
Worker1: Done 12000 out of 1303380 in 192.44 sec ~ 62.36/sec
Worker1: Done 13000 out of 1303380 in 199.53 sec ~ 65.15/sec
Worker1: Done 14000 out of 1303380 in 208.49 sec ~ 67.15/sec
Worker1: Done 15000 out of 1303380 in 216.17 sec ~ 69.39/sec
Worker1: Done 16000 out of 1303380 in 224.25 sec ~ 71.35/sec
Worker1: Done 17000 out of 1303380 in

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:67: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


Worker1: Done 50000 out of 1303380 in 505.18 sec ~ 98.98/sec
Worker1: Done 51000 out of 1303380 in 510.25 sec ~ 99.95/sec
Worker1: Done 52000 out of 1303380 in 514.08 sec ~ 101.15/sec
Worker1: Done 53000 out of 1303380 in 517.27 sec ~ 102.46/sec
Worker1: Done 54000 out of 1303380 in 522.12 sec ~ 103.42/sec
Worker1: Done 55000 out of 1303380 in 526.28 sec ~ 104.51/sec
Worker1: Done 56000 out of 1303380 in 534.19 sec ~ 104.83/sec
Worker1: Done 57000 out of 1303380 in 543.08 sec ~ 104.96/sec
Worker1: Done 58000 out of 1303380 in 553.10 sec ~ 104.86/sec
Worker1: Done 59000 out of 1303380 in 557.21 sec ~ 105.88/sec
Worker1: Done 60000 out of 1303380 in 566.92 sec ~ 105.83/sec
Worker1: Done 61000 out of 1303380 in 571.51 sec ~ 106.73/sec
Worker1: Done 62000 out of 1303380 in 593.37 sec ~ 104.49/sec
Worker1: Done 63000 out of 1303380 in 598.60 sec ~ 105.25/sec
Worker1: Done 64000 out of 1303380 in 605.48 sec ~ 105.70/sec
Worker1: Done 65000 out of 1303380 in 610.14 sec ~ 106.53/sec
Worker1: D

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:67: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/pymongo/topology.py:155: UserWarning: MongoClient opened before fork. Create MongoClient only after forking. See PyMongo's documentation for details: http://api.mongodb.org/python/current/faq.html#is-pymongo-fork-safe
  "MongoClient opened before fork. Create MongoClient only "


...

Worker1: Done 320000 out of 1303380 in 21016.70 sec ~ 15.23/sec
Worker1: Done 321000 out of 1303380 in 21189.15 sec ~ 15.15/sec
Worker2: Done 23000 out of 1303380 in 21190.92 sec ~ 1.09/sec
Worker1: Done 322000 out of 1303380 in 21261.16 sec ~ 15.14/sec
Worker1: Done 323000 out of 1303380 in 21349.61 sec ~ 15.13/sec
Worker1: Done 324000 out of 1303380 in 21437.71 sec ~ 15.11/sec
Worker1: Done 325000 out of 1303380 in 21637.12 sec ~ 15.02/sec
Worker3: Done 13000 out of 1303380 in 21747.47 sec ~ 0.60/sec
Worker1: Done 326000 out of 1303380 in 21791.15 sec ~ 14.96/sec
Worker2: Done 24000 out of 1303380 in 21856.77 sec ~ 1.10/sec
Worker1: Done 327000 out of 1303380 in 21974.01 sec ~ 14.88/sec
Worker1: Done 328000 out of 1303380 in 22107.53 sec ~ 14.84/sec
Worker1: Done 329000 out of 1303380 in 22188.66 sec ~ 14.83/sec
Worker1: Done 330000 out of 1303380 in 22358.58 sec ~ 14.76/sec
Worker1: Done 331000 out of 1303380 in 22568.92 sec ~ 14.67/sec
Worker1: Done 332000 out of 1303380 in 22729.3

Worker1: Done 412000 out of 1303380 in 34508.99 sec ~ 11.94/sec
Worker1: Done 413000 out of 1303380 in 34689.84 sec ~ 11.91/sec
Worker2: Done 41000 out of 1303380 in 34700.56 sec ~ 1.18/sec
Worker1: Done 414000 out of 1303380 in 34881.41 sec ~ 11.87/sec
Worker1: Done 415000 out of 1303380 in 34993.36 sec ~ 11.86/sec
Worker3: Done 23000 out of 1303380 in 35106.16 sec ~ 0.66/sec
Worker1: Done 416000 out of 1303380 in 35188.92 sec ~ 11.82/sec
Worker2: Done 42000 out of 1303380 in 35387.22 sec ~ 1.19/sec
Worker1: Done 417000 out of 1303380 in 35496.47 sec ~ 11.75/sec
Worker4: Done 16000 out of 1303380 in 35561.63 sec ~ 0.45/sec
Worker1: Done 418000 out of 1303380 in 35702.77 sec ~ 11.71/sec
Worker1: Done 419000 out of 1303380 in 35752.74 sec ~ 11.72/sec
Worker1: Done 420000 out of 1303380 in 35990.80 sec ~ 11.67/sec
Worker2: Done 43000 out of 1303380 in 36119.32 sec ~ 1.19/sec
Worker1: Done 421000 out of 1303380 in 36158.74 sec ~ 11.64/sec
Worker1: Done 422000 out of 1303380 in 36366.10 se

Worker1: Done 492000 out of 1303380 in 57512.94 sec ~ 8.55/sec
Worker4: Done 24000 out of 1303380 in 57590.14 sec ~ 0.42/sec
Worker2: Done 64000 out of 1303380 in 57609.37 sec ~ 1.11/sec
Worker1: Done 493000 out of 1303380 in 57924.22 sec ~ 8.51/sec
Worker1: Done 494000 out of 1303380 in 58380.73 sec ~ 8.46/sec
Worker3: Done 36000 out of 1303380 in 58741.81 sec ~ 0.61/sec
Worker2: Done 65000 out of 1303380 in 58832.09 sec ~ 1.10/sec
Worker1: Done 495000 out of 1303380 in 58867.54 sec ~ 8.41/sec
Worker1: Done 496000 out of 1303380 in 59407.48 sec ~ 8.35/sec
Worker1: Done 497000 out of 1303380 in 59921.79 sec ~ 8.29/sec
Worker2: Done 66000 out of 1303380 in 59994.81 sec ~ 1.10/sec
Worker1: Done 498000 out of 1303380 in 60406.65 sec ~ 8.24/sec
Worker4: Done 25000 out of 1303380 in 60659.88 sec ~ 0.41/sec
Worker3: Done 37000 out of 1303380 in 60953.12 sec ~ 0.61/sec
Worker1: Done 499000 out of 1303380 in 60979.93 sec ~ 8.18/sec
Worker2: Done 67000 out of 1303380 in 61046.87 sec ~ 1.10/sec


Worker1: Done 562000 out of 1303380 in 88958.26 sec ~ 6.32/sec
Worker2: Done 93000 out of 1303380 in 89391.03 sec ~ 1.04/sec
Worker1: Done 563000 out of 1303380 in 89528.84 sec ~ 6.29/sec
Worker1: Done 564000 out of 1303380 in 89873.44 sec ~ 6.28/sec
Worker3: Done 51000 out of 1303380 in 89916.84 sec ~ 0.57/sec
Worker1: Done 565000 out of 1303380 in 90226.22 sec ~ 6.26/sec
Worker2: Done 94000 out of 1303380 in 90292.31 sec ~ 1.04/sec
Worker1: Done 566000 out of 1303380 in 90553.72 sec ~ 6.25/sec
Worker1: Done 567000 out of 1303380 in 90896.87 sec ~ 6.24/sec
Worker2: Done 95000 out of 1303380 in 91010.23 sec ~ 1.04/sec
Worker4: Done 35000 out of 1303380 in 91172.17 sec ~ 0.38/sec
Worker1: Done 568000 out of 1303380 in 91180.59 sec ~ 6.23/sec
Worker3: Done 52000 out of 1303380 in 91425.62 sec ~ 0.57/sec
Worker5: Done 28000 out of 1303380 in 91430.18 sec ~ 0.31/sec
Worker1: Done 569000 out of 1303380 in 91468.18 sec ~ 6.22/sec
Worker1: Done 570000 out of 1303380 in 91758.06 sec ~ 6.21/sec

Worker1: Done 626000 out of 1303380 in 111426.03 sec ~ 5.62/sec
Worker3: Done 67000 out of 1303380 in 111718.61 sec ~ 0.60/sec
Worker1: Done 627000 out of 1303380 in 111753.01 sec ~ 5.61/sec
Worker2: Done 124000 out of 1303380 in 112032.39 sec ~ 1.11/sec
Worker1: Done 628000 out of 1303380 in 112061.84 sec ~ 5.60/sec
Worker1: Done 629000 out of 1303380 in 112408.49 sec ~ 5.60/sec
Worker4: Done 46000 out of 1303380 in 112453.25 sec ~ 0.41/sec
Worker1: Done 630000 out of 1303380 in 112725.23 sec ~ 5.59/sec
Worker2: Done 125000 out of 1303380 in 112772.08 sec ~ 1.11/sec
Worker1: Done 631000 out of 1303380 in 112915.84 sec ~ 5.59/sec
Worker3: Done 68000 out of 1303380 in 113041.79 sec ~ 0.60/sec
Worker5: Done 36000 out of 1303380 in 113113.80 sec ~ 0.32/sec
Worker5: Done 37000 out of 1303380 in 114545.60 sec ~ 0.32/sec
Worker2: Done 126000 out of 1303380 in 113432.23 sec ~ 1.11/sec
Worker1: Done 632000 out of 1303380 in 113303.16 sec ~ 5.58/sec
Worker3: Done 69000 out of 1303380 in 114002.

In [92]:
import os
import time

from pymongo import MongoClient
from nltk.stem.wordnet import WordNetLemmatizer

tags_collection = MongoClient(MONGO_CONNECTION_STRING)[TAGS_DATABASE][REVIEWS_COLLECTION]
corpus_collection = MongoClient(MONGO_CONNECTION_STRING)[TAGS_DATABASE][CORPUS_COLLECTION]

reviews_cursor = tags_collection.find()
reviewsCount = reviews_cursor.count()
reviews_cursor.batch_size(5000)

lem = WordNetLemmatizer()

done = 0
start = time.time()

for review in reviews_cursor:
    print(review['reviewId'])
    if review['reviewId'] in food_review_set:
        print(review['reviewId'],'found')
        food_review_record = food_review_df.loc[review['reviewId']]
        nouns = {word['word'] for word in review["words"] if word["pos"] in {"NN", "NNS"}}

        corpus_collection.insert({
            "reviewId": review["reviewId"],
            "business": review["business"],
            "text": review["text"],
            "words": list(nouns),
            "hours": food_review_record["hours"],
            "address": food_review_record["address"],
            "attributes":food_review_record["attributes"],
            "categories": food_review_record["categories"],
            "city": food_review_record["city"],
            "is_open":food_review_record["is_open"],
            "latitude": food_review_record["latitude"],
            "longitude": food_review_record["longitude"],
            "name": food_review_record["name"],
            "postal_code":food_review_record["postal_code"],
            "review_count":food_review_record["review_count"],
            "business_stars":food_review_record["business_stars"],
            "state":food_review_record["state"],
            "cool": food_review_record["cool"],
            "date": food_review_record["date"],
            "funny": food_review_record["funny"],
            "review_stars": food_review_record["review_stars"],
            "useful":food_review_record["useful"],
            "user_id":food_review_record["user_id"]
        })
        

    done += 1
    if done % 1000 == 0:
        end = time.time()
        os.system('cls')
        print('Done ' + str(done) + ' out of ' + str(reviewsCount) + ' in ' + str((end - start)))
    if done > 10:
        break
#     break

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  # This is added back by InteractiveShellApp.init_path()


GJXCdrto3ASJOqKeVWPi6Q
2TzJjDVDEuAW6MR5Vuc1ug
yi0R0Ugj_xUx_Nek0-_Qig
yi0R0Ugj_xUx_Nek0-_Qig found
11a8sVPMUFtaC7_ABRkmtw
fdiNeiN_hoCxCMy2wTRW9g
fdiNeiN_hoCxCMy2wTRW9g found
G7XHMxG0bx9oBJNECG4IFg
G7XHMxG0bx9oBJNECG4IFg found


/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:49: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


8e9HxxLjjqc9ez5ezzN7iQ
8e9HxxLjjqc9ez5ezzN7iQ found
qrffudO73zsslZbe8B9D3Q
qrffudO73zsslZbe8B9D3Q found
RS_GTIT6836bCaPy637kNQ
kbtscdyz6lvrtGjD1quQTg
kbtscdyz6lvrtGjD1quQTg found
-I5umRTkhw15RqpKMl_o1Q
-I5umRTkhw15RqpKMl_o1Q found


In [84]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient.get_database('Tags')
corpus_col = mydb.get_collection('Corpus')

mydb = myclient["Tags"]

In [59]:
thing = mydb.get_collection('Reviews').find_one()

In [144]:
corpus_col.count()

/Users/miraekim/miniconda3/envs/main_env/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

In [91]:
(corpus_col.find_one()['words'])

['server',
 'salad',
 'prices',
 'perfect',
 'sandwich',
 'experience',
 'drink',
 'steak',
 'dawn',
 'dressing',
 'lunch',
 'amount',
 'summation']

In [129]:
from gensim.models import Phrases
from gensim import corpora

def get_bigrams(docs):
    # Add bigrams and trigrams to docs (only ones that appear 20 times or more).
    bigram = Phrases(docs, min_count=20)
    for idx in range(len(docs)):
        for token in bigram[docs[idx]]:
            if '_' in token:
                # Token is a bigram, add to document.
                docs[idx].append(token)
    return docs



def build():
    cursor = corpus_col = mydb.get_collection('Corpus').find()
    input_text = get_bigrams([review["words"] for review in cursor])
    print(input_text)
    dictionary = corpora.Dictionary(input_text)
#     dictionary.filter_extremes(keep_n=20000)
#     dictionary.compactify()
    corpora.Dictionary.save(dictionary, '../test.dict')

    return dictionary

In [130]:
dict_ = build()

[['server', 'salad', 'prices', 'perfect', 'sandwich', 'experience', 'drink', 'steak', 'dawn', 'dressing', 'lunch', 'amount', 'summation'], ['server', 'salad', 'prices', 'perfect', 'sandwich', 'experience', 'drink', 'steak', 'dawn', 'dressing', 'lunch', 'amount', 'summation'], ['place', 'heard', 'desserts', 'job', 'things', 'tortellini', 'romano', 'im', 'house', 'recommend', 'wait', 'parm', 'friends', 'side', 'tavolta', 'jugs', 'teeny', 'waiter', 'bland', 'conversation', 'crab', 'dish', 'date', 'snob', 'pasta', 'room', 'people', 'la', 'group', 'sea', 'hearing', 'appetizers', 'die', 'chef', 'mussels', 'food', 'chicken', 'hubby', 'goodness', 'calamari', 'server', 'menu', 'reservation', 'service', 'table', 'dinner', 'alas', 'ordering', 'special', 'bass', 'bit', 'portion', 'order'], ['place', 'time', 'bowl', 'years', 'side', 'waitress', 'taste', 'dessert', 'tracy', 'balls', 'date', 'fancy', 'people', 'amount', 'taro', 'items', 'selection', 'friend', 'home', 'tapioca', 'chicken', 'wings', 'p

In [126]:
class Train:
    def __init__(self):
        pass

    @staticmethod
    def run(lda_model_path, corpus_path, num_topics, id2word):
        corpus = corpora.BleiCorpus(corpus_path)
        lda = gensim.models.LdaModel(corpus,
        num_topics=num_topics,
        id2word=id2word,
        )
        lda.save(lda_model_path)

        return lda

In [134]:
[x for x in dict_.items()]

[(0, 'amount'),
 (1, 'dawn'),
 (2, 'dressing'),
 (3, 'drink'),
 (4, 'experience'),
 (5, 'lunch'),
 (6, 'perfect'),
 (7, 'prices'),
 (8, 'salad'),
 (9, 'sandwich'),
 (10, 'server'),
 (11, 'steak'),
 (12, 'summation'),
 (13, 'alas'),
 (14, 'appetizers'),
 (15, 'bass'),
 (16, 'bit'),
 (17, 'bland'),
 (18, 'calamari'),
 (19, 'chef'),
 (20, 'chicken'),
 (21, 'conversation'),
 (22, 'crab'),
 (23, 'date'),
 (24, 'desserts'),
 (25, 'die'),
 (26, 'dinner'),
 (27, 'dish'),
 (28, 'food'),
 (29, 'friends'),
 (30, 'goodness'),
 (31, 'group'),
 (32, 'heard'),
 (33, 'hearing'),
 (34, 'house'),
 (35, 'hubby'),
 (36, 'im'),
 (37, 'job'),
 (38, 'jugs'),
 (39, 'la'),
 (40, 'menu'),
 (41, 'mussels'),
 (42, 'order'),
 (43, 'ordering'),
 (44, 'parm'),
 (45, 'pasta'),
 (46, 'people'),
 (47, 'place'),
 (48, 'portion'),
 (49, 'recommend'),
 (50, 'reservation'),
 (51, 'romano'),
 (52, 'room'),
 (53, 'sea'),
 (54, 'service'),
 (55, 'side'),
 (56, 'snob'),
 (57, 'special'),
 (58, 'table'),
 (59, 'tavolta'),
 (60,